# Brainstorming

Goal: Read my today twitter feed and extract the most important information. Then send it to my email with link for the important posts.

Steps:
1. [D] What important: new AI development or products, Tesla and stock market. Important news only. 
2. [D]Read twitter feed
3. [D]Extract important information
   1. [ ] Adjust prompt to get better results
4. Send email
5. Schedule it
6. Deploy it
7. Test it
8. Improve it






In [9]:
# !pip install tweepy

In [13]:
import tweepy
from dotenv import load_dotenv
import os

load_dotenv()

API_Key = os.getenv('twitter_api_key')
API_Key_Secret = os.getenv('twitter_api_keySecret')
access_token = os.getenv('twitter_access_token')
access_token_secret = os.getenv('twitter_access_token_secret')

# OAuth process, using the keys and tokens
auth = tweepy.OAuth1UserHandler(
   API_Key, API_Key_Secret, access_token, access_token_secret
)

auth.set_access_token(access_token, access_token_secret)

# Creation of the actual interface, using authentication
api = tweepy.API(auth)



In [16]:
import tweepy
from dotenv import load_dotenv
import os
import datetime
import openai
import string
load_dotenv()

openai.api_key = os.getenv('OPENAI_API_KEY')
API_Key = os.getenv('twitter_api_key')
API_Key_Secret = os.getenv('twitter_api_keySecret')
access_token = os.getenv('twitter_access_token')
access_token_secret = os.getenv('twitter_access_token_secret')

# OAuth process, using the keys and tokens
auth = tweepy.OAuth1UserHandler(API_Key, API_Key_Secret, access_token, access_token_secret)

# Creation of the actual interface, using authentication
api = tweepy.API(auth)

class TweetAnalyzer:
    def __init__(self):
        self.model_name = 'gpt-3.5-turbo'

    def is_important(self, text):
        prompt = """
        I'm interested in AI developments,.
        Is the tweet below important & related to my interest? (Only response "yes" or "no", no punctuation)):
        """
        
        response = openai.ChatCompletion.create(
            model=self.model_name,
            messages=[
                {"role": "user", "content": prompt + text}
            ]
        )
        decision = response['choices'][0]['message']['content'].lower()
        decision = decision.translate(str.maketrans("", "", string.punctuation)).strip()
        print(f"Decision: {decision}")
        return decision == "yes"

analyzer = TweetAnalyzer()

public_tweets = api.home_timeline(count=10, tweet_mode='extended')  # Maximum count is 200


today = datetime.date.today()

# Filter for tweets created today
today_tweets = [tweet for tweet in public_tweets if tweet.created_at.date() == today]


# Further filter for "important" tweets
important_tweets = []

for tweet in today_tweets:
    print(f"Date: {tweet.created_at}")
    print(f"Author: {tweet.user.name}")
    print(f"Tweet: {tweet.full_text}")
    print(f"URL: https://twitter.com/user/status/{tweet.id_str}")
    if analyzer.is_important(tweet.full_text):
        important_tweets.append(tweet)
    print("\n")
        
        
# Print the results
print(f"Collected {len(today_tweets)} tweets from today.")
print(f"Found {len(important_tweets)} important tweets.")

for tweet in important_tweets:
    print(f"Date: {tweet.created_at}")
    print(f"Author: {tweet.user.name}")
    print(f"Tweet: {tweet.full_text}")
    print(f"URL: https://twitter.com/user/status/{tweet.id_str}\n")

Date: 2023-05-04 06:24:00+00:00
Author: Sawyer Merritt
Tweet: NEWS: SpaceX to surpass 4,000 Starlink satellites on-orbit with Starlink Group 5-6.

This mission will mark SpaceX’s 27th Falcon 9 flight of the year. The company has launched a rocket on average every ~4.20 days in 2023.
https://t.co/09neqwkxhr
URL: https://twitter.com/user/status/1654008957197860866
Decision: no


Date: 2023-05-04 06:22:51+00:00
Author: Sawyer Merritt
Tweet: NEWS: Run On Less electric truck performance event will give inside look at the operations of PepsiCo's Tesla Semi Class 8 tractor.
https://t.co/begGZ1PEYK
URL: https://twitter.com/user/status/1654008669514727424
Decision: yes


Date: 2023-05-04 06:12:01+00:00
Author: Morten Just
Tweet: https://t.co/px3ZK77lTN from @NiklasQuarfot and team comes close. Curious what the playground looks like! https://t.co/yrXjBpV7Nu
URL: https://twitter.com/user/status/1654005941602246657
Decision: yes


Date: 2023-05-04 06:04:08+00:00
Author: Morten Just
Tweet: Found th

# Email the important tweets to me

In [21]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_email(subject, body, to_email, from_email, password):
    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = to_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(from_email, password)
        text = msg.as_string()
        server.sendmail(from_email, to_email, text)
        server.quit()
        print("Email sent successfully.")
    except Exception as e:
        print(f"Error sending email: {e}")

# Set up your email credentials
from_email = os.getenv('your_email')
password = os.getenv('your_email_password')
to_email = os.getenv('recipient_email')


# Prepare the email content
subject = "Important Tweets"
body = "Here are the important tweets:\n\n"
for tweet in important_tweets:
    body += f"Date: {tweet.created_at}\n"
    body += f"Author: {tweet.user.name}\n"
    body += f"Tweet: {tweet.full_text}\n"
    body += f"URL: https://twitter.com/user/status/{tweet.id_str}\n\n"

# Send the email
print(f"Password: {password}")
send_email(subject, body, to_email, from_email, password)

Password: Xuanthanh0611$
Error sending email: (534, b'5.7.9 Application-specific password required. Learn more at\n5.7.9  https://support.google.com/mail/?p=InvalidSecondFactor e5-20020a17090a630500b002478d21de2bsm10431617pjj.36 - gsmtp')
